## Concurrency
[The Why, When, and How of Using Python Multi-threading and Multi-Processing](https://medium.com/towards-artificial-intelligence/the-why-when-and-how-of-using-python-multi-threading-and-multi-processing-afd1b8a8ecca)

### Single-threaded, single-process

In [11]:
import urllib.request
from concurrent.futures import ThreadPoolExecutor

In [12]:
urls = [
  'http://www.python.org',
  'https://docs.python.org/3/',
  'https://docs.python.org/3/whatsnew/3.7.html',
  'https://docs.python.org/3/tutorial/index.html',
  'https://docs.python.org/3/library/index.html',
  'https://docs.python.org/3/reference/index.html',
  'https://docs.python.org/3/using/index.html',
  'https://docs.python.org/3/howto/index.html',
  'https://docs.python.org/3/installing/index.html',
  'https://docs.python.org/3/distributing/index.html',
  'https://docs.python.org/3/extending/index.html',
  'https://docs.python.org/3/c-api/index.html',
  'https://docs.python.org/3/faq/index.html'
  ]

In [13]:
%%time
results = []
for url in urls:
    with urllib.request.urlopen(url) as src:
        results.append(src)

CPU times: user 167 ms, sys: 8.12 ms, total: 175 ms
Wall time: 5.42 s


### Multi-threading


In [14]:
%%time

with ThreadPoolExecutor(4) as executor:
    results = executor.map(urllib.request.urlopen, urls)

CPU times: user 152 ms, sys: 22.5 ms, total: 175 ms
Wall time: 1.08 s


In [15]:
%%time

with ThreadPoolExecutor(8) as executor:
    results = executor.map(urllib.request.urlopen, urls)

CPU times: user 149 ms, sys: 21.1 ms, total: 170 ms
Wall time: 435 ms


In [16]:
%%time

with ThreadPoolExecutor(16) as executor:
    results = executor.map(urllib.request.urlopen, urls)

CPU times: user 167 ms, sys: 27.9 ms, total: 195 ms
Wall time: 450 ms


### Multi-processing

In [17]:
from multiprocessing import Pool

In [18]:
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

In [19]:
%%time

answer = 0

for i in range(1000000):
    answer += if_prime(i)

CPU times: user 3.2 s, sys: 0 ns, total: 3.2 s
Wall time: 3.21 s


In [20]:
%%time

if __name__ == '__main__':
    with Pool(2) as p:
        answer = sum(p.map(if_prime, list(range(1000000))))

CPU times: user 100 ms, sys: 55.2 ms, total: 155 ms
Wall time: 1.8 s


In [21]:
%%time

if __name__ == '__main__':
    with Pool(4) as p:
        answer = sum(p.map(if_prime, list(range(1000000))))

CPU times: user 108 ms, sys: 28.3 ms, total: 136 ms
Wall time: 949 ms


In [17]:
%%timeit

if __name__ == '__main__':
    with Pool(8) as p:
        answer = sum(p.map(if_prime, list(range(1000000))))

777 ms ± 41.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit

if __name__ == '__main__':
    with Pool(16) as p:
        answer = sum(p.map(if_prime, list(range(1000000))))

770 ms ± 45.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit

if __name__ == '__main__':
    with Pool(32) as p:
        answer = sum(p.map(if_prime, list(range(1000000))))

862 ms ± 65.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 1115. Print FooBar Alternately

Suppose you are given the following code:
```
class FooBar {
  public void foo() {
    for (int i = 0; i < n; i++) {
      print("foo");
    }
  }

  public void bar() {
    for (int i = 0; i < n; i++) {
      print("bar");
    }
  }
}
```

The same instance of FooBar will be passed to two different threads:

    thread A will call foo(), while
    thread B will call bar().

Modify the given program to output "foobar" n times.

 

Example 1:

Input: n = 1
Output: "foobar"

Explanation: There are two threads being fired asynchronously. One of them calls foo(), while the other calls bar().
"foobar" is being output 1 time.

Example 2:

Input: n = 2
Output: "foobarfoobar"

Explanation: "foobar" is being output 2 times.

 

Constraints:

    1 <= n <= 1000


In [4]:
import threading

class FooBar:
    def __init__(self, n):
        self.n = n

    def foo(self, printFoo: 'Callable[[], None]') -> None:
        for i in range(self.n):
            # printFoo() outputs "foo". Do not change or remove this line.
            printFoo()

    def bar(self, printBar: 'Callable[[], None]') -> None:
        for i in range(self.n):
            # printBar() outputs "bar". Do not change or remove this line.
            printBar()

n = 5

foo_bar = FooBar(n)


def print_foo():
    print("foo")


def print_bar():
    print("bar")


t1 = threading.Thread(target=foo_bar.foo, args=(print_foo,))
t2 = threading.Thread(target=foo_bar.bar, args=(print_bar,))

t1.start()
t2.start()

t1.join()
t2.join()

foo
foo
bar
bar
bar
bar
bar
foo
foo
foo


In [10]:
class FooBar:
    def __init__(self, n):
        self.n = n
        # 1. Khởi tạo lock: foo_lock và bar_lock đều ở trạng thái unlocked
        self.foo_lock = threading.Lock()
        self.bar_lock = threading.Lock()
        # 2. Call acquire() --> khóa bar_lock lại, trạng thái của bar_lock là locked
        self.bar_lock.acquire()

    def foo(self, printFoo: 'Callable[[], None]') -> None:
        for i in range(self.n):
            # 3.1.1 Trạng thái foo_lock đang là unlocked, code được thực thi tiếp 
            # sau đó chuyển trạng thái foor_lock sang locked
            self.foo_lock.acquire()
            # 3.2. printFoo được thực thi
            printFoo()  # Critical Section
            # 3.3. release bar_lock: chuyển trạng thái bar_lock sang unlocked -> đi đến bước 3.4
            self.bar_lock.release()

    def bar(self, printBar: 'Callable[[], None]') -> None:
        for i in range(self.n):
            # 3.1.2 Trạng thái bar_lock đang là locked, block thực thi cho đến khi bar_lock call release()
            self.bar_lock.acquire()
            # 3.4 bar_lock ở trạng thái unlocked 
            # printBar được thực thi và chuyển trạng thái bar_lock sang locked
            printBar() # Critical Section
            # 3.5. release foo_lock: chuyển trạng thái foo_lock sang unlocked -> đi đến bước 3.1.1
            self.foo_lock.release()


n = 5
foo_bar = FooBar(n)


def print_foo():
    print("foo")


def print_bar():
    print("bar")


t1 = threading.Thread(target=foo_bar.foo, args=(print_foo,))
t2 = threading.Thread(target=foo_bar.bar, args=(print_bar,))

t1.start()
t2.start()

t1.join()
t2.join()


foo
bar
foo
bar
foo
bar
foo
bar
foo
bar
